In [37]:
import os
import glob
import pandas as pd

In [38]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/Snai2Intensity'

In [39]:
# Add experiment date here to apply to dataframe
analysis_date = '20181110'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe


for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label
    df['label'], df['ROI'] = zip(*df['Label'].map(lambda x: x.split(':')))
    # Split values in Image name column
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'], 
        df['Somites'], df['ImageMag'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,IntDen,RawIntDen,Image,label,ROI,ExptDate,Treatment,Stains,Embryo,Somites,ImageMag,Section
0,1,Background2a:background,17.829,14.370,256.202,4972.0,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb6_8ss_40x...,Background2a,background,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb6,8ss,40x,sec3
1,2,Background2b:background,9.430,14.366,135.470,2629.0,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb6_8ss_40x...,Background2b,background,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb6,8ss,40x,sec3
2,3,Background2c:background,13.449,13.490,181.434,3521.0,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb6_8ss_40x...,Background2c,background,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb6,8ss,40x,sec3
3,4,CntlSide2:Cntl,4535.944,51.501,233605.882,4533483.0,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb6_8ss_40x...,CntlSide2,Cntl,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb6,8ss,40x,sec3
4,5,ExptSide2:Expt,4597.830,47.120,216649.077,4204410.0,20181102_SMPD3gRNA1RNP_Snai2Cad6b_Emb6_8ss_40x...,ExptSide2,Expt,20181102,SMPD3gRNA1RNP,Snai2Cad6b,Emb6,8ss,40x,sec3


In [40]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Mean background values and group by Treatment, Embryo, Fluor, ROI and Section
mean_sections = ((full_df.groupby(['Treatment', 'Embryo', 'ROI', 'Section', 'ExptDate'])
                  ['Area', 'Mean', 'IntDen']).mean())

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(mean_sections.xs(treatment))

    # Determine CTCF values = ROI IntDen - (background mean * ROI area)
    # Calculate background (background mean * ROI area)
    background_corr_cntl = (df_treatment.xs('background', level='ROI')['Mean'] 
                    * df_treatment.xs('Cntl', level='ROI')['Area'])
    background_corr_expt = (df_treatment.xs('background', level='ROI')['Mean'] 
                    * df_treatment.xs('Expt', level='ROI')['Area'])

    # Slice out only Cntl or Expt values in IntDen
    intdens_cntl = df_treatment.xs('Cntl', level='ROI')['IntDen'] 
    intdens_expt = df_treatment.xs('Expt', level='ROI')['IntDen'] 

    # Subtract background from IntDens to determine CTCF and concatenate into single dataframe
    sub_cntl = pd.DataFrame(intdens_cntl - background_corr_cntl)
    sub_expt = pd.DataFrame(intdens_expt - background_corr_expt)
    full_ctcf = pd.concat([sub_cntl, sub_expt], keys = ['Cntl', 'Expt'])
    full_ctcf.columns = ['CTCF']

    # Average sections grouped by embryos before generating Expt/Cntl ratio
    full_ctcf.index.names = ['Side', 'Embryo', 'Section', 'ExptDate']
    averaged_sections = full_ctcf.groupby(['Side','Embryo', 'ExptDate']).mean()

    # Pull out Cntl and Expt CTCFs
    ctcf_cntl = averaged_sections.xs('Cntl', level='Side')['CTCF'] 
    ctcf_expt = averaged_sections.xs('Expt', level='Side')['CTCF'] 

    # Generate ratios as Expt/Cntl
    ratios_sections = pd.DataFrame(ctcf_expt / ctcf_cntl)
    ratios_sections.columns = ['Expt/Cntl CTCF']

    # Normalize individual values to mean of control group
    norm_cntl = ctcf_cntl/(float(ctcf_cntl.mean()))
    norm_cntl = pd.DataFrame(norm_cntl)
    norm_cntl.columns = ['Norm Cntl CTCF']
    norm_expt = ctcf_expt/(float(ctcf_cntl.mean()))
    norm_expt = pd.DataFrame(norm_expt)
    norm_expt.columns = ['Norm Expt CTCF']

    # Combine processed values into single dataframe and output as csv file 'Results.csv'
    ctcf_cntl = pd.DataFrame(ctcf_cntl)
    ctcf_cntl.columns = ['Cntl CTCF']
    ctcf_expt = pd.DataFrame(ctcf_expt)
    ctcf_expt.columns = ['Expt CTCF']
    results = (pd.concat([ctcf_cntl, ctcf_expt, ratios_sections, norm_cntl, norm_expt], axis=1, sort=True)).reset_index()
    results['ID'] = results.ExptDate.str.cat(results.Embryo)
    results.to_csv(analysis_date + '_' + treatment + '_Snai2IntensityResults.csv')